In [ ]:
from pathlib import Path
import sys

def configure_gpu_memory(gpu_index: int) -> bool:
    """
    Activa el crecimiento de memoria dinámico en las GPUs.
    
    Args:
        gpu_index: Índice de la GPU a exponer (0, 1, …). Si es None, se configura en todas.
    Returns:
        True si la configuración se aplicó correctamente, False en caso de error o si no hay GPUs.
    """
    import os
    import tensorflow as tf
    # Listar GPUs físicas
    gpus = tf.config.list_physical_devices('GPU')
    if not gpus:
        print("❌ No se encontraron GPUs disponibles.")
        return False

    try:
        # Si el usuario especifica un índice, solo exponemos esa GPU
        if gpu_index is not None:
            tf.config.set_visible_devices(gpus[gpu_index], 'GPU')
            gpus = [gpus[gpu_index]]

        # Para cada GPU visible, activar crecimiento de memoria
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        objetivo = f"GPU {gpu_index}" if gpu_index is not None else "todas las GPUs"
        print(f"✅ Crecimiento de memoria dinámico activado en {objetivo}.")
        return True

    except RuntimeError as e:
        # Ocurre si TensorFlow ya inicializó las GPUs antes de esta llamada
        print(f"❌ Error configurando GPUs: {e}")
        return False
    
def configure_gpu(gpu_index=1, memory_limit=4096*4):
    import os
    import tensorflow as tf

    gpus = tf.config.list_physical_devices('GPU')
    print('gpus: ', gpus)
    if gpus:
        try:
            # Limitar visibilidad a la GPU deseada
            tf.config.set_visible_devices(gpus[gpu_index], 'GPU')

            # Crear un dispositivo lógico con límite de memoria
            tf.config.set_logical_device_configuration(
                gpus[gpu_index],
                [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
            )

            print(f"✅ GPU {gpu_index} configurada correctamente con {memory_limit} MB.")
            return True
        except RuntimeError as e:
            print(f"❌ Error configurando GPU: {e}")
            return False
    else:
        print("❌ No se encontraron GPUs disponibles.")
        return False
    

# configure_gpu_memomry(gpu_index=0)
configure_gpu(gpu_index=0)


BASE_DIR     = Path().resolve()
MODELS_DIR   = BASE_DIR / 'models'

# =============================================================================
# 🚀 Notebook principal de experimentación con TensorFlow/Keras
# =============================================================================

# 1) Preparar el entorno
sys.path.append(str(MODELS_DIR))
sys.path.append(str(BASE_DIR))

# 2) Definir experimento
#    El nombre debe coincidir con un archivo YAML en configs/experiments/
exp_names = ["< EXP >"]


for exp_name in exp_names:
  # 3) Cargar configuración
  from utils.experiment.functions import load_config, load_experiment
  cfg = load_config(exp_name)

  # 4) Repeticiones
  repeats = cfg["experiment"].get("repeats")

  for rep in range(repeats):

      #  4.1) Cargar experimento
      cfg, NNClass, params, dataset, train_data, val_data, test_idx = \
          load_experiment(exp_name, repeat_index=rep)

      #  4.1.1) Revisar que 'rep' actual no haya sido previamente ejecutado
      #   Si ya existe classification_report.json  → SALTAR
      rep_report = BASE_DIR / cfg['experiment']['output_root'] / cfg['experiment']['output_subdir'] / "reports" / "classification_report.json"
      if rep_report.exists() == True:
        print(f"[SKIP] Rep: {rep} (single‐split) → ya existe classification_report.json.")
        continue

      #  4.1.2) Instanciar y Entrenar
      model   = NNClass(cfg, **params)

      if rep == 0:
        #  4.1.3 ) Mostrar resumen de la configuración
        # from utils.misc.functions import print_exp_configuration
        print(f"\n✔️ Experimento «{cfg['experiment']['name']}» cargado con éxito.\n")
        # print_exp_configuration(cfg)

        #  4.1.4) Mostrar arquitectura del modelo
        print("\n📋 Arquitectura del modelo:")
        model.model.summary()

      print("\n"*5)
      print(f"\n🔄 Rep {rep+1}/{repeats}")

      #  4.1.5) Entrenamiento (o retomar desde último checkpoint)
      history = model.fit(train_data, val_data)

      #  4A.6) Análisis resultados individual
      from utils.analysis.analysis import ExperimentAnalyzer
      analyzer = ExperimentAnalyzer(
         model=model.model,
         history=history,
         test_data=test_idx,
         cfg=cfg,
         effects=dataset.get_effects("test"),
         repeat_index=rep,
         show_plots=False,
         )

      analyzer.classification_report()
      analyzer.effect_report()
      analyzer.confusion_matrix(normalize="true")
      model.cleanup_old_checkpoints()

  print("\n"*2)
  print("="*15, f" | EXPERIMENTO '{exp_name}' FINALIZADO CORRECTAMENTE  | ", "="*15)
  print("\n"*3)